### Importando Bibliotecas a serem utilizadas

In [ ]:
import pandas as pd
#import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from statsmodels.tsa.seasonal import seasonal_decompose #decomposição
# from statsmodels.tsa.stattools import adfuller #teste Estatistico de Fuller (PValue)
# from statsmodels.tsa.stattools import acf, pacf #Auto Correlação (ACF) Auto correlação Parcial (PACF)
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# import yfinance as yf #Dados Da Bolsa importado diretamente do Python
# from bcb import sgs, currency # Dados do banco central
# import ipeadatapy as idpy

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
%pip install panda

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7244 sha256=c90bbc5c192582a0a8e27c566dc90f6471561ff82b025c33be5e6f1ceb2d7223
  Stored in directory: c:\users\jeici\appdata\local\pip\cache\wheels\6c\5a\a9\a28c87238ddcc61701224f538bedccc21c9e2b8bc6fa8fca0e
Successfully built panda
Note: you may need to restart the kernel to use updated packages.


## Importando a Base de Dados da IBOVESPA e índice VIX

Utilizando a biblioteca Python 'yfinance' que possui os dados diários através do link(https://finance.yahoo.com/quote/%5EBVSP/history/)

O Yahoo Finance utiliza 5 colunas relativas a preço do ativos:

Open -> Preço de abertura do dia do ativo (sem ajuste).

High -> Maior preço alcançado no dia pelo ativo (sem ajuste).

Low -> Menor preço alcançado no dia pelo ativo (sem ajuste).

Close -> Preço de fechamento do dia do ativo (sem ajuste).

Adj Close -> Preço de fechamento do dia do ativo (com ajuste) ##  Com o preço de fechamento, você verá apenas a mudança bruta no preço, enquanto o preço de fechamento ajustado também levará em consideração o dividendo. Para ações que não pagam dividendos, não haverá diferença entre esses dois valores.

Volume -> Número de negócios que teve no dia.

In [ ]:
#Dados Da Bolsa importado diretamente do Python
start_date = '2021-05-06'
end_date = '2024-07-25'

df_origem = yf.download('^BVSP', start=start_date, end=end_date)

In [ ]:
df_origem.head()

In [ ]:
# Indice S&P/B3 Ibovespa VIX -> Avaliação de risco
df_vix = pd.read_excel('PerformanceGraphExport.xlsx')

In [ ]:
df_vix = df_vix.set_index('Data efetiva')
df_vix.head()

In [ ]:
#Taxa Selic

selic_df = sgs.get(('selic', 432), start = '2001-01-01')

selic_df.head()

In [ ]:
#Criando um df com o indice vix
df_novo = df_origem.merge(df_vix, how = 'left',  left_on = df_origem.index, right_on = df_vix.index)
df_novo.rename(columns={'key_0':'Date'}, inplace = True)
df_novo = df_novo.set_index('Date')
df_novo.head()

## Análise Exploratória de Dados

In [ ]:
df_novo.info()

In [ ]:
#Quantidade de duplicados
print('Volume de duplicados na amostra: ', df_novo.duplicated().sum())

In [ ]:
#Estatistica descritiva
df_novo.describe() 

In [ ]:
# Fechamento
fig, ax1 = plt.subplots(figsize = (15, 10))
ax1.plot(df_novo.index, df_novo['Adj Close'], color='blue', label='Close Price')
ax1.set_xlabel('Date')
ax1.set_ylabel('Close', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# VIX
ax2 = ax1.twinx()
ax2.plot(df_novo.index, df_novo['S&P/B3 Ibovespa VIX'], color='green', label='HS&P/B3 Ibovespa VIX')
ax2.set_ylabel('S&P/B3 Ibovespa VIX', color='green')
ax2.tick_params(axis='y', labelcolor='green')

plt.title('Fechamento x Indice VIX')
plt.show()

In [ ]:
#Cria a matriz de correlação
correlation_matrix = df_novo.corr()

fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(data=correlation_matrix, annot=True, linewidths=.5, ax=ax)

In [ ]:
x = df_novo[["Open", "High", "Low", "S&P/B3 Ibovespa VIX"]].values #variaveis independentes
y = df_novo["Close"].values #target (alvo)

In [ ]:
sns.scatterplot(data=df_novo, x='S&P/B3 Ibovespa VIX', y='Close')

In [ ]:
ax = df_novo.plot.hist(column=[""], by="gender", figsize=(10, 8))

In [ ]:
#Plot com os dados de Fechamento da Bolsa
# fig, ax = plt.subplots(figsize= (14, 14))
plt.plot(df_origem.index, df_origem['Close'])
plt.title('IBOVESPA - Fechamento')
plt.xlabel('Data')
plt.ylabel('Fechamento')
# plt.show()

### Análise dos dados

##Decomposição

In [ ]:
##Duvida ##Não entendi pq tenho q definir um periodo

resultados = seasonal_decompose(df,period=365)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1, figsize=(15,10))

resultados.observed.plot(ax=ax1)
#Tendecia
resultados.trend.plot(ax=ax2)
#Sazonalidade
resultados.seasonal.plot(ax=ax3)
#Residuo
resultados.resid.plot(ax=ax4)

plt.tight_layout()

##Teste Estatístico - Fuller

In [ ]:
X = df.y.values

In [ ]:
results = adfuller(X)

print("Teste ADF")
print(f"Teste Estatistico: {results[0]}")
print(f"P-Value: {results[1]}")
print("Valores criticos:")

for key, value in results[4].items():
  print(f"{key}: {value}")

Criando Média Móvel (MA)

In [ ]:
ma = df.rolling(12).mean()

f, ax = plt.subplots()
df.plot(ax=ax, legend=False)
ma.plot(ax=ax, legend=False, color='r')
plt.tight_layout()

In [ ]:
#Aplica Logaritimo para remover as medias moveis

df_log = np.log(df)
ma_log = df_log.rolling(12).mean()

f, ax = plt.subplots()
df_log.plot(ax=ax, legend=False)
ma_log.plot(ax=ax, legend=False, color='r')
plt.tight_layout()

In [ ]:
df_s = (df_log - ma_log).dropna() #dropa os nulos

ma_s = df_s.rolling(12).mean()

std = df_s.rolling(12).std()

f, ax = plt.subplots()
df_s.plot(ax=ax, legend=False)
ma_s.plot(ax=ax, legend=False, color='r')
std.plot(ax=ax, legend=False, color='g')
plt.tight_layout()


In [ ]:
X_s = df_s.y.values


results_S = adfuller(X_s)

print("Teste ADF")
print(f"Teste Estatistico: {results_S[0]}")
print(f"P-Value: {results_S[1]}")
print("Valores criticos:")

for key, value in results_S[4].items():
  print(f"{key}: {value}")

In [ ]:
#Aplicar a diferenciação para suavizar ainda mais a serie estacionaria


df_diff = df_s.diff(1)  #1 -- primeira derivada
ma_diff = df_diff.rolling(12).mean()

std_diff = df_diff.rolling(12).std()

f, ax = plt.subplots()
df_diff.plot(ax=ax, legend=False)
ma_diff.plot(ax=ax, legend=False, color='r')
std_diff.plot(ax=ax, legend=False, color='g')
plt.tight_layout()


In [ ]:
X_diff = df_diff.y.dropna().values
results_diff = adfuller(X_diff)

print("Teste ADF")
print(f"Teste Estatistico: {results_diff[0]}")
print(f"P-Value: {results_diff[1]}")
print("Valores criticos:")

for key, value in results_diff[4].items():
  print(f"{key}: {value}")

In [ ]:
plot_acf(df.y)
plot_pacf(df.y)
plt.show()